Creates binary masks for all images for semantic segmentation. Also fill faulty holes in masks.

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
from scipy import ndimage
import numpy as np
import pickle
from cytoolz import partition

In [ ]:
data_dir = Path("/kaggle/input/sartorius-cell-instance-segmentation")
output_dir = Path("/kaggle/working")

In [ ]:
D=pd.read_csv(data_dir.joinpath("train.csv"))
D.head()

In [ ]:
def rle2mask(rle, width, height):
    result = np.zeros(width * height, dtype=np.uint8)
    for start, length in rle:
        start -= 1
        result[start : start + length] = 1
    return result.reshape(height, width)


def row2mask(row):
    return rle2mask(partition(2, map(int, row.annotation.split())), row.width, row.height)


img_masks = {}
mask_dir=output_dir.joinpath("train_masks")
mask_dir.mkdir(exist_ok=True)

for id_, rows in tqdm(D.groupby("id")):
    masks = [ndimage.binary_fill_holes(row2mask(row)).astype(np.uint8) for row in rows.itertuples(False)]
    full_mask = np.max(masks, axis=0)
    with mask_dir.joinpath(f"mask_{id_}.pkl").open("wb") as f:
        pickle.dump(full_mask, f)